### Graph based Music Recommender. Task 5

#### Task description
Data description (DataFrames in parquet format)
Location - /data/sample264

Fields: trackId, userId, timestamp, artistId

trackId - id of the track
userId - id of the user
artistId - id of the artist
timestamp - timestamp of the moment the user starts listening to a track
Location - /data/meta

Fields: type, Name, Artist, Id

Type could be “track” or “artist”
Name is the title of the track if the type == “track” and the name of the musician or group if the type == “artist”.
Artist states for the creator of the track in case the type == “track” and for the name of the musician or group in case the type == “artist”.
Id - id of the item
Task
Construct balancing function where the edges of the type “user-track” and the edges of the type “user-artist” influence the final recommendations equally.

For the user with Id 776748 find all the tracks and artists connected to him. Sort founded items first by artist then by name in ascending order, leave only columns ”Artist” and “Name” and print top-40

In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [3]:
window = Window.partitionBy('userId')

user_to_track = data.groupBy('userId', col('trackId').alias('id')).count()
user_to_artist = data.groupBy('userId', col('artistId').alias('id')).count()

result = user_to_track.union(user_to_artist) \
                      .join(meta, on='id') \
                      .withColumn('sum_count', sum('count').over(window)) \
                      .withColumn('norm_count', col('count') / col('sum_count') * 0.5) \
                      .filter(col('userId') == 776748) \
                      .select(col('Artist'), col('Name')) \
                      .orderBy(col('Artist'), col('Name')) \
                      .take(40)

In [4]:
for val in result:
    print "%s %s" % val

Artist: 3 Doors Down Artist: 3 Doors Down
Artist: 3 Doors Down Kryptonite
Artist: 311 Artist: 311
Artist: 311 Beautiful disaster
Artist: Blur Artist: Blur
Artist: Blur Girls and Boys
Artist: Clawfinger Artist: Clawfinger
Artist: Clawfinger Nothing Going On
Artist: Disturbed Artist: Disturbed
Artist: Disturbed The Vengeful One
Artist: Gotthard Artist: Gotthard
Artist: Gotthard Eagle
Artist: Green Day 21 Guns
Artist: Green Day Artist: Green Day
Artist: Green Day Kill The DJ
Artist: Iggy Pop Artist: Iggy Pop
Artist: Iggy Pop Sunday
Artist: Korn Artist: Korn
Artist: Korn Here To Stay
Artist: Linkin Park Artist: Linkin Park
Artist: Linkin Park In The End
Artist: Linkin Park Numb
Artist: Lordi Artist: Lordi
Artist: Lordi Hard Rock Hallelujah
Artist: Nickelback Artist: Nickelback
Artist: Nickelback She Keeps Me Up
Artist: Nomy Artist: Nomy
Artist: Nomy Cocaine
Artist: Papa Roach Artist: Papa Roach
Artist: Papa Roach Getting Away With Murder
Artist: Rise Against Artist: Rise Against
Artist: Ri